In [20]:
import json
import pickle
import numpy as np
import torch
import pandas as pd
from utils import Vocab
from typing import Dict
from tqdm import trange
from pathlib import Path
from model import SeqClassifier,SeqTagger
from dataset import SeqTaggingClsDataset
from torch.utils.data import DataLoader
from argparse import ArgumentParser, Namespace
from IPython.display import display

In [2]:
cache_dir = Path("./cache/slot/")
data_dir = Path("./data/slot/")
ckpt_dir = Path("./ckpt/slot/")
max_len = 128
batch_size = 512
hidden_size = 512
num_layers = 2
dropout = 0.1
bidirectional = True
device = "cuda"
num_epoch = 50
lr = 1e-3

In [3]:
with open(cache_dir / "vocab.pkl", "rb") as f:
    vocab: Vocab = pickle.load(f)

In [4]:
tag_idx_path = cache_dir / "tag2idx.json"
tag2idx: Dict[str, int] = json.loads(tag_idx_path.read_text())

In [5]:
TRAIN = "train"
DEV = "eval"
SPLITS = [TRAIN, DEV]

In [6]:
# create datasets
data_paths = {
    split: data_dir / f"{split}.json" 
    for split in SPLITS
}
data = {
    split: json.loads(path.read_text()) 
    for split, path in data_paths.items()
}


data_train = [
    {
        "token": token,
        "tag": tag,
        "id": item["id"]
        
    }
    for item in data["train"]
    for token, tag in zip(item["tokens"], item["tags"])
]


data_eval = [
    {
        "token": token,
        "tag": tag,
        "id": item["id"]
        
    }
    for item in data["eval"]
    for token, tag in zip(item["tokens"], item["tags"])
]

data = {
    "train": data_train,
    "eval": data_eval
}

datasets: Dict[str, SeqTaggingClsDataset] = {
    split: SeqTaggingClsDataset(split_data, vocab, tag2idx, max_len)
    for split, split_data in data.items()
}
display(datasets)

{'token': 'i', 'tag': 'O', 'id': 'train-0'}
{'token': 'i', 'tag': 'O', 'id': 'eval-0'}


{'train': <dataset.SeqTaggingClsDataset at 0x7fbab617cd90>,
 'eval': <dataset.SeqTaggingClsDataset at 0x7fbab60808e0>}

In [7]:
train_data_loader = DataLoader(
    datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=datasets["train"].collate_fn
)
dev_data_loader = DataLoader(
    datasets["eval"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=datasets["eval"].collate_fn,
)
# for batch in train_data_loader:
#     print(batch["ids"])
#     print(batch["labels"])
#     print(batch["index"])

In [8]:
embeddings = torch.load(cache_dir / "embeddings.pt")
# TODO: init model and move model to target device(cpu / gpu)
model = SeqTagger(
    embeddings,
    hidden_size,
    num_layers,
    dropout,
    bidirectional,
    len(tag2idx),
)
model.to(device)

# TODO: init optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []
best_valid_loss = float('inf')

epoch_pbar = trange(num_epoch, desc="Epoch")
for epoch in epoch_pbar:
    # TODO: Training loop - iterate over train dataloader and update model weights
    model.train()
    epoch_train_losses = []
    epoch_train_accs = []
    for batch in train_data_loader:
        ids = batch["ids"].to(device)
        labels = batch["labels"].to(device)
        pred = model(ids).to(device)
        loss = criterion(pred, labels).to(device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_train_losses.append(loss.item())
        epoch_train_accs.append((pred.argmax(dim=1) == labels).float().mean().item())

    # TODO: Evaluation loop - calculate accuracy and save model weights
    model.eval()
    epoch_eval_losses = []
    epoch_eval_accs = []
    with torch.no_grad():
        for batch in dev_data_loader:
            ids = batch["ids"].to(device)
            labels = batch["labels"].to(device)
            pred = model(ids).to(device)
            loss = criterion(pred, labels).to(device)
            epoch_eval_losses.append(loss.item())
            epoch_eval_accs.append((pred.argmax(dim=1) == labels).float().mean().item())

    train_losses.extend(epoch_train_losses)
    train_accs.extend(epoch_train_accs)
    valid_losses.extend(epoch_eval_losses)
    valid_accs.extend(epoch_eval_accs)

    epoch_train_loss = np.mean(epoch_train_losses)
    epoch_train_acc = np.mean(epoch_train_accs)
    epoch_valid_loss = np.mean(epoch_eval_losses)
    epoch_valid_acc = np.mean(epoch_eval_accs)

    print(epoch_train_acc)
    print(epoch_valid_acc)

    if epoch_valid_loss < best_valid_loss:
        best_valid_loss = epoch_valid_loss
        torch.save(model.state_dict(), ckpt_dir / 'slot_model.pt')

Epoch:   2%|███▎                                                                                                                                                                  | 1/50 [01:27<1:11:05, 87.05s/it]

0.8390431729230013
0.8616312779486179


Epoch:   4%|██████▋                                                                                                                                                               | 2/50 [02:56<1:10:51, 88.57s/it]

0.8626970410346985
0.8703179396688938


Epoch:   6%|█████████▉                                                                                                                                                            | 3/50 [04:27<1:10:18, 89.77s/it]

...

0.8922754927115006
0.883445993065834


Epoch:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 49/50 [1:14:53<01:33, 93.62s/it]

0.891784461519935
0.883690133690834


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [1:16:26<00:00, 91.73s/it]

0.8931953310966492
0.8829594478011131


# Predict model

In [5]:
embeddings = torch.load(cache_dir / "embeddings.pt")
cache_dir_dir = Path("./cache/slot/")
test_file = Path("data/slot/test.json")
with open(cache_dir / "vocab.pkl", "rb") as f:
    vocab: Vocab = pickle.load(f)

intent_idx_path = cache_dir / "tag2idx.json"
intent2idx: Dict[str, int] = json.loads(intent_idx_path.read_text())

data = json.loads(test_file.read_text())

data_eval = [
    {
        "token": token,
        "id": item["id"]   
    }
    for item in data
    for token in item["tokens"]
]

dataset = SeqTaggingClsDataset(data_eval, vocab, intent2idx, max_len)

# TODO: crecate DataLoader for test dataset
test_data_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=dataset.collate_fn,
)

In [13]:
test_model = SeqTagger(
    embeddings,
    hidden_size,
    num_layers,
    dropout,
    bidirectional,
    len(tag2idx),
)

test_model.load_state_dict(torch.load("./ckpt/slot/slot_model.pt"))
test_model.to(device)
test_model.eval()

result_dict = {}
result_index_list = list()
result_ids_list = list()

for batch in test_data_loader:
    ids = batch["ids"].to(device)
    pred = test_model(ids).to(device)
    
#     print("-" * 59)
#     print(pred.argmax(dim=1))
    
#     print("-" * 59)
#     print(pred.argmax(dim=1).shape)
    
#     print("-" * 59)
#     print(batch["index"])
    
#     print("-" * 59)
#     print(len(batch["index"]))
          
#     print("-" * 59)

    result_ids_list.extend(
        pred.argmax(dim=1).tolist()
    )
    
    result_index_list.extend(batch["index"])

In [14]:
display(len(result_ids_list))
display(len(result_index_list))

28571

28571

In [15]:
for index, id in zip(result_index_list, result_ids_list):
    if index not in result_dict:
        result_dict[index] = []
    result_dict[index].append(
        dataset.idx2label(id)
    )

In [24]:
csv_dict = {
    "id": [],
    "tags": []
}
for k, v in result_dict.items():
    csv_dict["id"].append(k)
    csv_dict["tags"].append(
        " ".join(v)
    )

csv_result = pd.DataFrame(csv_dict)
csv_result.to_csv("slot_res.csv", index=False)

In [26]:
csv_result.head()

,id,tags
0,test-0,O O O O B-people O O B-time
1,test-1,O O O O O O O B-people I-date O B-time O
2,test-2,O O O O O B-people
3,test-3,O O O O O
4,test-4,O O O O


In [27]:
csv_result.iloc[60, :]

id                                   test-60
tags    O O O I-people I-people O O I-people
Name: 60, dtype: object